# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.

## Descripción de los datos

Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la columna `num_orders`.

## Preparación

### Importación de librerías y creación de variables globales

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

test_size = 0.1 # tamaño del conjunto de prueba
rs = 12345 # semilla de random state
min_rmse = 48.0 # RMSE objetivo

In [2]:
# Función de valoración de los modelos.
def valoracion(model_rmse, model_name):
    print("El modelo de {} posee un RMSE de {}".format(model_name,model_rmse))
    if model_rmse < min_rmse:
        return "El modelo cumple con el mínimo RMSE de {}.".format(min_rmse)
    return "El modelo no cumple con el mínimo RMSE de {}".format(min_rmse)

### Importación de datos

In [3]:
# Leemos los datos.
df = pd.read_csv(
    "https://practicum-content.s3.us-west-1.amazonaws.com/datasets/taxi.csv?etag=11687de0e23962e5a11c9d8ae13eb630",
    index_col=[0], parse_dates=[0])
df.sort_index(inplace=True)
df.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


## Análisis

La tabla solamente posee el índice en tipo fecha/hora y la columna num_orders. A continuación se analizarán sus propiedades.

### Descripción de los datos

In [4]:
# Verificamos el tipo de dato de las columnas y verifiamos que no haya valores faltantes.
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


In [5]:
# Vemos la distribución de los datos.
df.describe()

,num_orders
count,26496.000000
mean,14.070463
std,9.211330
min,0.000000
25%,8.000000
50%,13.000000
75%,19.000000
max,119.000000


In [6]:
# Verificamos el rango del índice.
# La tabla comprende datos entre marzo y agosto de 2018.
print("Mínimo: ",df.index.min())
print("Máximo: ",df.index.max())

Mínimo:  2018-03-01 00:00:00
Máximo:  2018-08-31 23:50:00


In [7]:
# Remuestreo por una hora
df = df.resample("60min").sum()
df.head()

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43


In [8]:
# Verificamos el rango del índice.
print("Mínimo: ",df.index.min())
print("Máximo: ",df.index.max())

Mínimo:  2018-03-01 00:00:00
Máximo:  2018-08-31 23:00:00


## Formación

### Creación de características

A continuación se crearán diversas características a partir de la fecha y hora.

In [9]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data["hour"] = data.index.hour

    # Columnas con retraso.
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    # Promedio móvil.
    data['rolling_mean'] = (
        data['num_orders'].shift().rolling(rolling_mean_size).mean()
    )

In [10]:
make_features(df,6,10)
df.head()

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,rolling_mean
datetime,,,,,,,,,,,,,
2018-03-01 00:00:00,124,2018,3,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 01:00:00,85,2018,3,1,3,1,124.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 02:00:00,71,2018,3,1,3,2,85.0,124.0,NaN,NaN,NaN,NaN,NaN
2018-03-01 03:00:00,66,2018,3,1,3,3,71.0,85.0,124.0,NaN,NaN,NaN,NaN
2018-03-01 04:00:00,43,2018,3,1,3,4,66.0,71.0,85.0,124.0,NaN,NaN,NaN


In [11]:
df.describe()

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,rolling_mean
count,4416.000000,4416.0,4416.000000,4416.000000,4416.000000,4416.00000,4415.000000,4414.000000,4413.000000,4412.000000,4411.000000,4410.000000,4406.000000
mean,84.422781,2018.0,5.505435,15.836957,3.005435,11.50000,84.395470,84.364069,84.347156,84.331369,84.319655,84.291837,84.316841
std,45.023853,0.0,1.713306,8.855229,1.990684,6.92297,44.992356,44.949043,44.940088,44.932942,44.931299,44.898387,29.695361
min,0.000000,2018.0,3.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.000000
25%,54.000000,2018.0,4.000000,8.000000,1.000000,5.75000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,62.600000
50%,78.000000,2018.0,5.500000,16.000000,3.000000,11.50000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,80.200000
75%,107.000000,2018.0,7.000000,23.250000,5.000000,17.25000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,100.700000
max,462.000000,2018.0,8.000000,31.000000,6.000000,23.00000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,213.400000


In [12]:
# Función para codificar los atributos cícliclos en seno y coseno.
# De esta manera los modelos pueden captar la naturaleza cíclica de columnas como dayofweek y month.
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
    return data

In [13]:
encode(df,"month",12)
encode(df,"day",31)
encode(df,"dayofweek",7)
encode(df,"hour",24)
df.sample(5)

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,...,lag_6,rolling_mean,month_sin,month_cos,day_sin,day_cos,dayofweek_sin,dayofweek_cos,hour_sin,hour_cos
datetime,,,,,,,,,,,,,,,,,,,,,
2018-08-24 10:00:00,205,2018,8,24,4,10,170.0,95.0,38.0,68.0,...,93.0,116.1,-8.660254e-01,-0.500000,-0.988468,0.151428,-0.433884,-0.900969,0.500000,-8.660254e-01
2018-04-01 18:00:00,53,2018,4,1,6,18,50.0,100.0,50.0,44.0,...,33.0,55.8,8.660254e-01,-0.500000,0.201299,0.979530,-0.781831,0.623490,-1.000000,-1.836970e-16
2018-06-30 08:00:00,103,2018,6,30,5,8,24.0,22.0,39.0,146.0,...,151.0,109.8,1.224647e-16,-1.000000,-0.201299,0.979530,-0.974928,-0.222521,0.866025,-5.000000e-01
2018-05-26 09:00:00,89,2018,5,26,5,9,84.0,35.0,7.0,34.0,...,136.0,104.7,5.000000e-01,-0.866025,-0.848644,0.528964,-0.974928,-0.222521,0.707107,-7.071068e-01
2018-07-25 16:00:00,134,2018,7,25,2,16,119.0,95.0,99.0,83.0,...,95.0,89.7,-5.000000e-01,-0.866025,-0.937752,0.347305,0.974928,-0.222521,-0.866025,-5.000000e-01


In [14]:
# Retiramos las columnas originales
df.drop(["month","day","dayofweek","hour"],axis=1,inplace=True)

In [15]:
# Dividimos los datos en conjuntos de entrenamiento y prueba.
from sklearn.model_selection import train_test_split

df2 = df.dropna()
X = df2.drop("num_orders",axis=1)
y = df2["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state = rs, shuffle=False)


## Prueba de Modelos

### Regresión Lineal

In [16]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)
lr_preds = lr.predict(X_test)
lr_rmse = mean_squared_error(y_test,lr_preds,squared=False)
valoracion(lr_rmse, "Regresión Lineal")

El modelo de Regresión Lineal posee un RMSE de 53.14063605805457


'El modelo no cumple con el mínimo RMSE de 48.0'

### Árbol de Decisión

In [17]:
from sklearn.tree import DecisionTreeRegressor

dt_best_model = None
dt_best_result = float("inf")
dt_best_depth = 0

In [18]:
%%time
for depth in range(1, 40): # selecciona el rango del hiperparámetro
    model = DecisionTreeRegressor(random_state=rs,max_depth=depth) # entrena el modelo en el conjunto de entrenamiento
    model.fit(X_train,y_train) # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(X_test) # obtén las predicciones del modelo en el conjunto de validación
    result = mean_squared_error(y_test,predictions_valid,squared=False) # calcula la RECM en el conjunto de validación
    if result < dt_best_result:
        dt_best_model = model
        dt_best_result = result
        dt_best_depth = depth

#print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {dt_best_depth}): {dt_best_result}")
valoracion(dt_best_result,"Árbol de Decisión")

El modelo de Árbol de Decisión posee un RMSE de 50.26122596675444
CPU times: user 1.09 s, sys: 1.01 ms, total: 1.09 s
Wall time: 1.09 s


'El modelo no cumple con el mínimo RMSE de 48.0'

### Bosque Aleatorio

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf_best_score = float("inf")
rf_best_est = 0
rf_best_depth = 0
rf_best_model = None

In [20]:
%%time
# Seleccionamos el rango del hiperparámetro.
for est in range(10, 50,10):
    for depth in range(5,30):
    
        # Iniciamos el constructor.
        model = RandomForestRegressor(random_state=rs, n_estimators=est, max_depth = depth)
    
        # Entrenamos el modelo en el conjunto de entrenamiento.
        model.fit(X_train, y_train)
    
        # Realizamos predicciones en el conjunto de validación
        preds = model.predict(X_test)
    
        # Calculamos la exactitud.
        #score = model.score(features_val,target_val)
    
        # Calculamos el RMSE
        score = mean_squared_error(y_test,preds,squared=False)
    
        # Guardamos la cantidad de estimadores que generaron el mejor puntaje.
        if score < rf_best_score:
            rf_best_model = model
            rf_best_score = score
            rf_best_est = est
            rf_best_depth = depth

#print("El RMSE del mejor modelo en el conjunto de validación (n_estimators = {}, max_depth = {}): {}".format(
#    rf_best_est, rf_best_depth, rf_best_score))
print("Mejores Hiperparámetros:")
print("    Estimators: ",rf_best_est)
print("    Depth: ",rf_best_depth)
valoracion(rf_best_score,"Bosque Aleatorio")

Mejores Hiperparámetros:
    Estimators:  10
    Depth:  23
El modelo de Bosque Aleatorio posee un RMSE de 43.91842916242346
CPU times: user 41.8 s, sys: 156 ms, total: 42 s
Wall time: 42 s


'El modelo cumple con el mínimo RMSE de 48.0.'

## Conclusión

En el presente proyecto se realizaron distintos modelos con el objetivo de encontrar a uno que pueda predecir el número de órdenes en la serie de tiempo proporcionada por la empresa Swift Lift Taxi. Luego de probar varios modelos se encontró un modelo de Bosque Aleatorio que cumple con un RMSE menor al objetivo de 48. El resultado fue el siguiente:

| Modelo | RMSE |
|---|---|
| Bosque Aleatorio | 43.91 |
| Árbol de Decisión | 50.26 |
| Regresión Lineal | 53.14 |

Esperamos seguir contando con la confianza de Swift Lift Taxi para futuros proyectos.